#Dependencies#

In [ ]:
!pip install transformers
!pip install datasets transformers[SentencePiece]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

FOLDERNAME = 'CS685-Project'

import sys
sys.path.append('/content/drive/My Drive/{}/code'.format(FOLDERNAME))

%load_ext autoreload
%autoreload 2

%cd /content/drive/My\Drive/$FOLDERNAME

Mounted at /content/drive
[Errno 2] No such file or directory: '/content/drive/MyDrive/CS685-Project'
/content


In [ ]:
from datasets import *
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, AdamW, DataCollatorWithPadding, get_scheduler
import pandas as pd
import torch
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from solver import *
from models import *
from rmse import *
from utils import *
from log_control import *
from transformers import logging
from datetime import date
date = date.today()
model_lib = {}
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')